### PREDICTING CRSP (CURRENT RETAIL SELLING PRICE) FOR KRA AUTOMOBILE DATA

### IDE (Initial Data Exploration)

In [266]:
# Mathematical Operations Library
import numpy as np

# Data Manipulation Library
import pandas as pd

# Visualisation Libraries
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import seaborn as sns 

# Statistical Libraries
import statsmodels.api as sm

# Machine Learning Libraries
import scipy.stats as stats
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

# Filter warnings
import warnings
warnings.filterwarnings('ignore')

In [267]:
# Load and print out the data
df = pd.read_excel('KRA Vehicle Data.xlsx', sheet_name = 'M.Vehicle CRSP July 2025', header = 1)

df

,Make,Model,Model \nnumber,Transmission,Drive\nConfiguration,Engine \nCapacity,Body \nType,GVW,Seating,Fuel,CRSP (KES.)
0,AIWAYS,U5,NaN,AT,FWD,63 kWh,SUV,2155,5,ELECTRIC,5.910345e+06
1,AIWAYS,U6,NaN,AT,FWD,63 kWh,SUV,1820,5,ELECTRIC,8.232266e+06
2,AUDI,3.0TFSI QUATTRO S LINE PACKAGE,WAUZZZ4M8JD01,AT,4WD,3000,SUV,NaN,NaN,GASOLINE,1.621063e+07
3,AUDI,A1 SPORTBACK 1.0 TFSI,DBA-8XCHZ,AT,2WD,1000,HATCHBACK,NaN,NaN,GASOLINE,5.953578e+06
4,AUDI,A1 SPORTBACK 1ST EDITION,3BA-GBDAD,AT,2WD,1500,HATCHBACK,NaN,NaN,GASOLINE,8.573479e+06
...,...,...,...,...,...,...,...,...,...,...,...
5274,XPENG,G9 RWD LONG RANGE,NaN,AUT,RWD,93.1 kWh,NaN,2285 kg,5,ELECTRIC,1.300276e+07
5275,XPENG,G9 RWD STANDARD RANGE,NaN,AUT,RWD,75.8 kWh,NaN,2310 kg,5,ELECTRIC,1.215842e+07
5276,XPENG,P7 AWD PERFORMANCE,NaN,AUT,AWD,82.7 kWh,NaN,2215 kg,5,ELECTRIC,1.236951e+07
5277,XPENG,P7 RWD LONG RANGE,NaN,AUT,RWD,82.7 kWh,NaN,2095 kg,5,ELECTRIC,1.046975e+07


In [268]:
# Check shape
print(f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns.")

The dataset has 5279 rows and 11 columns.


In [269]:
# Print out column names
df.columns

Index(['Make', 'Model', 'Model \nnumber', 'Transmission',
       'Drive\nConfiguration', 'Engine \nCapacity', 'Body \nType ', 'GVW',
       'Seating', 'Fuel', 'CRSP (KES.)'],
      dtype='object')

These columns do not look clean. They are readable but they could use some cleaning. We will remove the '\n' and replace the space with an underscore.

In [270]:
# Standardise columns
df.columns = df.columns.str.strip().str.lower().str.replace('\n', '_')

# Rename columns
df = df.rename(columns = 
                       {
                           'model _number': 'model_number',
                           'engine _capacity': 'engine_capacity',
                           'body _type': 'body_type',
                           'crsp (kes.)': 'crsp_in_kes'
                       })
df.columns

Index(['make', 'model', 'model_number', 'transmission', 'drive_configuration',
       'engine_capacity', 'body_type', 'gvw', 'seating', 'fuel',
       'crsp_in_kes'],
      dtype='object')

In [271]:
# Get metadata
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5279 entries, 0 to 5278
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make                 5279 non-null   object 
 1   model                5279 non-null   object 
 2   model_number         3437 non-null   object 
 3   transmission         5231 non-null   object 
 4   drive_configuration  4754 non-null   object 
 5   engine_capacity      5212 non-null   object 
 6   body_type            5220 non-null   object 
 7   gvw                  1708 non-null   object 
 8   seating              2943 non-null   object 
 9   fuel                 5263 non-null   object 
 10  crsp_in_kes          5277 non-null   float64
dtypes: float64(1), object(10)
memory usage: 453.8+ KB


> #### *Observation:*
> The dataset contains largely categorical variables (Dtype: 'object') with the exception of crsp_in_kes which is the only numerical variable, which has monetary values in Kenyan Shillings

Next we get basic descriptive statistical information about these variables. First we start with categorical columns then the numeric column

In [272]:
# Get summary statistics of all categorical features
df.describe(include = 'O').T

,count,unique,top,freq
make,5279,66,LANDROVER,479
model,5279,4073,RANGER,27
model_number,3437,2347,MAN,31
transmission,5231,66,AT,1763
drive_configuration,4754,49,2WD,2607
engine_capacity,5212,273,2000,702
body_type,5220,45,SUV,1058
gvw,1708,357,1090,25
seating,2943,24,5,1523
fuel,5263,21,GASOLINE,2351


In [273]:
# Get summary statistics of numeric feature
df.describe()

,crsp_in_kes
count,5.277000e+03
mean,1.377391e+07
std,1.832609e+07
min,5.444018e+05
25%,4.879632e+06
50%,8.882243e+06
75%,1.569003e+07
max,4.239280e+08


In [274]:
# Check duplicates
print("Duplicates:", df.duplicated().sum())

# Check missing values with counts & percentages
null_counts =df.isna().sum()
null_percentages = (null_counts / len(df)) * 100

print("\nMissing Values Overview:\n")
print(pd.concat([null_counts.rename("Null Count"), null_percentages.round(2).rename("Null Percentage (%)")], axis = 1))

Duplicates: 12

Missing Values Overview:

                     Null Count  Null Percentage (%)
make                          0                 0.00
model                         0                 0.00
model_number               1842                34.89
transmission                 48                 0.91
drive_configuration         525                 9.95
engine_capacity              67                 1.27
body_type                    59                 1.12
gvw                        3571                67.65
seating                    2336                44.25
fuel                         16                 0.30
crsp_in_kes                   2                 0.04


In [275]:
# Drop duplicates
df.drop_duplicates(inplace = True)
print("Duplicates after removal:", df.duplicated().sum())

Duplicates after removal: 0


### DATA UNDERSTANDING

What do all these columns mean?

### DATA CLEANING

We start with the easiest one to clean: crsp_in_kes. We will impute this with the median value

In [ ]:
# Imputing with median
df['crsp_in_kes'] = df['crsp_in_kes'].fillna(df['crsp_in_kes'].median())

# Sanity check
print(f"Missing values in 'crsp_in_kes': {df['crsp_in_kes'].isna().sum()}")

Missing values in 'crsp_in_kes': 0


Next we deal with the nulls in the fuel column. But first, let's explore the column. See what values lie within.

In [277]:
df['fuel'].unique()

array(['ELECTRIC', 'GASOLINE', 'HYBRID', 'DIESEL', 'PLUG-IN HYBRID',
       'PLUG-IN HYBRID ', 'ELECCTRIC', 'ELECTRIC(EV)', 'PETROL',
       'PLUG-IN-HYBRID', nan, ' DIESEL', 'CNG', 'LNG', 'PETROL/ELECTRIC',
       ' PETROL', 'DEISEL', 'PETROL/DIESEL', '40', '37', 'DIESE L',
       'DI ESEL'], dtype=object)

The fuel column is a mess: inconsistent casing, typos, stray numbers, random separators.  
Before we even think of imputing, we need to normalize this chaos.  

Game Plan
1. Strip whitespace + enforce consistent casing  
   Electric instead of ELECTRIC, Diesel instead of Diese L.  
2. Fix typos / misspellings  
   Elecctric to EV, Deisel to Diesel.  
3. Standardize labels  
   - Electric to EV  
   - Plug-In Hybrid & variants to Plug-in Hybrid  
   - Gasoline to Petrol  
4. Handle oddballs 
   - Petrol/Diesel to Dual Fuel  
   - Petrol/Electric to Hybrid  
   - "37", "40" to 'Unknown'

In [278]:
# Define normalization map
fuel_map = {
    "Electric": "EV",
    "Elecctric": "EV",
    "Electric(Ev)": "EV",
    "Gasoline": "Petrol",
    "Deisel": "Diesel",
    "Diese L": "Diesel",
    "Di Esel": "Diesel",
    "Plug-In-Hybrid": "Plug-In Hybrid",
    "Petrol/Electric": "Hybrid",
    "Petrol/Diesel": "Dual Fuel",
    "Cng": "CNG",
    "Lng": "LNG",
    "40": "Unknown",
    "37": "Unknown"
}

# Clean fuel column
df['fuel'] = df['fuel'].str.strip().str.title().replace(fuel_map)

# Sanity check
df['fuel'].unique()

array(['EV', 'Petrol', 'Hybrid', 'Diesel', 'Plug-In Hybrid', nan, 'CNG',
       'LNG', 'Dual Fuel', 'Unknown'], dtype=object)

Perfect! Now we check the value counts for these variables so we know what metric we will use for imputation

In [279]:
df['fuel'].value_counts()

fuel
Petrol            3014
Diesel            1511
EV                 356
Hybrid             292
Plug-In Hybrid      71
CNG                  2
LNG                  2
Unknown              2
Dual Fuel            1
Name: count, dtype: int64

We will impute using the mode

In [281]:
# Imputing with mode
df['fuel'] = df['fuel'].fillna(df['fuel'].mode()[0])

# Sanity check
print(f"Missing values in 'fuel': {df['fuel'].isna().sum()}")

Missing values in 'fuel': 0


We will drop the Unknown category because it offers no analytical value whatsoever. We do not have a fuel called Unknown

In [282]:
# Dropping Unknown feature in fuel column
df['fuel'].drop(df[df['fuel'] == 'Unknown'].index, inplace=True)

# Sanity check
print(f"Missing values in 'fuel': {df['fuel'].isna().sum()}")

Missing values in 'fuel': 0


Now let's deal with body_type

In [283]:
df['body_type'].unique()

array(['SUV', 'HATCHBACK', 'SEDAN', 'WAGON', 'CONVERTIBLE', 'COUPE',
       'CONVRTIBLE', 'suv', 'PICKUP', 'CROSSOVER', 'D/CAB', 'S. WAGON',
       'VAN', 'TRUCK', 'MINIVAN', 3, 'TRK', 'S/CAB', 'DOUBLE  CAB',
       'TIPPER', 'MIXER', 'BUS', 'SAL', 'PM', 'STATION WAGON',
       'DOUBLE CABIN', 'SINGLE CABIN', 'PEOPLE MOVER', 'DUAL CAB',
       'CREW CAB', 'OTHER', 'S/CABIN', nan, 'SINGLE CAB', 'S/WAGON',
       'MINVAN', 'SUV-COUPE', 'ROADSTER', 'TRACTOR', 'HATCBACK',
       'PICK UP', 'TRANSIT  MIXER', 'PRIM£ MOVER', 'AMBULANCE',
       'MINI BUS', 'SALOON'], dtype=object)